In [1]:
import requests
import pandas as pd
import json
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
DATA_REPRESENTATION = 2
DOWNSAMPLE_FACTOR = 5
WINDOW_LENGHT =  1
LIMITADOR = 20000
LOSS_FACTOR = .96
TRAIN_SIZE = 0.8
PATH_DATASET = '../dataset/original/'

In [3]:
url = 'http://127.0.0.1:5000/detector'
headers = {'Accept' : 'application/json', 'Content-Type' : 'application/json'}

def send(endpoint, params_dict):
    return requests.get(url+endpoint, data=json.dumps(params_dict), headers=headers).json()

In [4]:
dict_ds_original = {
    'data_ds3_normal_t1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t1.csv', header=None),
    'data_ds3_normal_t2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t2.csv', header=None),
    'data_ds3_fault1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault1_leakage.csv', header=None),
    'data_ds3_fault2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault2_viscousfriction.csv', header=None),
    'data_ds3_fault3_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault3_compressibility.csv', header=None),
    'data_ds3_fault4_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault4_fixedposition.csv', header=None),
}

dict_ds = dict_ds_original.copy()

if dict_ds['data_ds3_normal_t1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0 or dict_ds['data_ds3_fault1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0:
    raise Exception('Needs to be ?shape? divisor')

for n, dataset_name in enumerate(dict_ds):
    dataset = dict_ds[dataset_name].to_numpy()

    downsampled = dataset[::DOWNSAMPLE_FACTOR]

    x, y = downsampled.shape

    # resample
    dict_ds[dataset_name] = pd.DataFrame(downsampled.reshape((int(x/WINDOW_LENGHT),y*WINDOW_LENGHT)))

# ADD COLUMNS WITH DIFF PREVIOUS VALUES

if (DATA_REPRESENTATION == 2):
    frame_size = int(1000/DOWNSAMPLE_FACTOR)

    for n, dataset_name in enumerate(dict_ds):
        dataset = dict_ds[dataset_name].to_numpy()

        dimension = dataset.shape[1]
        samples = dataset.shape[0]

        # GENERATE NEW DIMENSIONS
        dataset = np.concatenate((dataset, np.zeros((samples,dimension))), axis=1)

        for f in np.arange(0,int(samples/frame_size)):
            # OBTAIN THE FRAME FLIGHT
            frame = dataset[f*frame_size:(f+1)*frame_size, 0:dimension]

            # CALCULATE DIFFERENCE
            chunk = np.diff(frame, axis=0)

            # DONT CALCULATE THE DIFFERENCE FOR EACH FIRST TIMESTEP
            chunk = np.insert(chunk, 0, frame[0, 0:dimension], axis=0)

            # UPDATE DATASET WITH NEW FRAME INTO NEW DIMENSIONS
            dataset[f*frame_size:(f+1)*frame_size,dimension:dimension*2] = chunk

        dict_ds[dataset_name] = pd.DataFrame(dataset)

ss = StandardScaler()

data_ds3_t1_normal = dict_ds['data_ds3_normal_t1_original']
data_ds3_t2_normal = dict_ds['data_ds3_normal_t2_original']
data_ds3_fault1 = dict_ds['data_ds3_fault1_original']
data_ds3_fault2 = dict_ds['data_ds3_fault2_original']
data_ds3_fault3 = dict_ds['data_ds3_fault3_original']
data_ds3_fault4 = dict_ds['data_ds3_fault4_original']

# fit values
ss.partial_fit(data_ds3_t1_normal)
ss.partial_fit(data_ds3_t2_normal)
ss.partial_fit(data_ds3_fault1)
ss.partial_fit(data_ds3_fault2)
ss.partial_fit(data_ds3_fault3)
ss.partial_fit(data_ds3_fault4)

# transform values
data_ds3_t1_normal = ss.transform(data_ds3_t1_normal)
data_ds3_t2_normal = ss.transform(data_ds3_t2_normal)
data_ds3_fault1 = ss.transform(data_ds3_fault1)
data_ds3_fault2 = ss.transform(data_ds3_fault2)
data_ds3_fault3 = ss.transform(data_ds3_fault3)
data_ds3_fault4 = ss.transform(data_ds3_fault4)

# append normal labels
data_ds3_t1_normal = np.append(data_ds3_t1_normal, np.zeros((data_ds3_t1_normal.shape[0],1)), axis = 1)
data_ds3_t2_normal = np.append(data_ds3_t2_normal, np.zeros((data_ds3_t2_normal.shape[0],1)), axis = 1)

# append fault labels
def generate_fault_label(dataset, fault_label):
    labels = np.array([[fault_label]]*dataset.shape[0])

    return labels

data_ds3_fault1 = np.append(data_ds3_fault1, generate_fault_label(data_ds3_fault1, 1), axis = 1)
data_ds3_fault2 = np.append(data_ds3_fault2, generate_fault_label(data_ds3_fault2, 2), axis = 1)
data_ds3_fault3 = np.append(data_ds3_fault3, generate_fault_label(data_ds3_fault3, 3), axis = 1)
data_ds3_fault4 = np.append(data_ds3_fault4, generate_fault_label(data_ds3_fault4, 4), axis = 1)

In [5]:
_, test_n1, _, _ = train_test_split(data_ds3_t1_normal[:, :-1], data_ds3_t1_normal[:, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

_, test_n2, _, _ = train_test_split(data_ds3_t2_normal[:, :-1], data_ds3_t2_normal[:, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

_, test_f1, _, _ = train_test_split(data_ds3_fault1[:, :-1], data_ds3_fault1[:, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

_, test_f2, _, _ = train_test_split(data_ds3_fault2[:, :-1], data_ds3_fault2[:, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

_, test_f3, _, _ = train_test_split(data_ds3_fault3[:, :-1], data_ds3_fault3[:, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

_, test_f4, _, _ = train_test_split(data_ds3_fault4[:, :-1], data_ds3_fault4[:, -1], test_size=1-TRAIN_SIZE, random_state=42, shuffle=False)

In [14]:
y_n1 = []
y_n2 = []
y_f1 = []
y_f2 = []
y_f3 = []
y_f4 = []

for _, x in enumerate(test_n1):
    response = send('/predict', {'x': pd.DataFrame(x).T.to_json()})
    y_n1.append(response['y_hat'])
    print(response['y_hat'], end='')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
for _, x in enumerate(test_n2):
    response = send('/predict', {'x': pd.DataFrame(x).T.to_json()})
    y_n2.append(response['y_hat'])
    print(response['y_hat'], end='')

In [23]:
for _, x in enumerate(test_f1):
    response = send('/predict', {'x': pd.DataFrame(x).T.to_json()})
    y_f1.append(response['y_hat'])
    print(response['y_hat'], end='')

1111111111111111111111111111111110000000000000000000111111111111111111111111111111111111111000000111000000111111111111111111111111111111111111111110000111000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110000001111111111111111111111111111111111111111111111110000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000011111100000111111111111111111111111111111111111111111111111111111111111111111111111111111000111111111111111111111111111111111111110000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000000011111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111100000000000000000001111111111111111111111111111111111111111100000000000000000000011111111111111111111110111111111111111111111111111111111111111111111111111111111111111

KeyboardInterrupt: 

In [145]:
for _, x in enumerate(test_f2):
    response = send('/predict', {'x': pd.DataFrame(x).T.to_json()})
    y_f2.append(response['y_hat'])
    print(response['y_hat'], end='')

111111111111111111111111111111

KeyboardInterrupt: 

In [146]:
for _, x in enumerate(test_f3):
    response = send('/predict', {'x': pd.DataFrame(x).T.to_json()})
    y_f3.append(response['y_hat'])
    print(response['y_hat'], end='')

111001101110111111111111011101101000011010000011001110010110101011101100110100111

KeyboardInterrupt: 

In [147]:
for _, x in enumerate(test_f4):
    response = send('/predict', {'x': pd.DataFrame(x).T.to_json()})
    y_f4.append(response['y_hat'])
    print(response['y_hat'], end='')

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

KeyboardInterrupt: 